In [62]:
import PySimpleGUI as sg
import openpyxl as ex
from pulp import *
import pandas as pd
import numpy as np
import math
import string
import os, sys
import datetime as dt




file1=open("parameters.txt","r")
param=file1.readlines()[0].split()

for i in range(len(param)):
    param[i]=int(param[i])
file1.close()

 
file1=open("parameters.txt","r")    
y={}
z=0
comp_values=file1.readlines()[1].split()
for i in range(param[1]):
    for j in range(param[2]):
        y[(i+1,j+1)]=int(comp_values[z])
        z+=1


file1.close()



size=30
log=["bilkent", "1947"]
SBattery = range(1, (1+param[1]))
SCar = range(1, (1+param[2]))
SHour = range(0,size)
SHourExt = range(0,size+2)
Demand = {t: [10,6,16] for t in SHour}



numOfBatteries = np.full((param[1],), 60, dtype=int)
Init_Inventory = numOfBatteries.tolist()
init_produced = np.full((3,param[1]), [None]*param[1])
Init_ProductionMix = init_produced.tolist()

numOfBatteriesLot = np.full((param[1],), 40, dtype=int)
Init_Inventory_Lot = numOfBatteriesLot.tolist()
init_producedLot = np.full((4,param[1]), [None]*param[1])
Init_ProductionLot = init_producedLot.tolist()






def solver():
    
    
    prob = LpProblem('InvenTri', LpMinimize)
    
    SBattery = range(1, (1+param[1]))
    SCar = range(1, (1+param[2]))
    SHour = range(0,size)
    SHourExt = range(0,size+2)

      
    

    



    P = LpVariable.dicts("P", (SBattery, SHour), lowBound = 0, cat='Integer')
    I = LpVariable.dicts("I", (SBattery, SHour), lowBound = 0, cat='Integer')
    G = LpVariable.dicts("G", (SBattery, SCar, SHourExt), lowBound = 0, cat='Integer')
    
    
    for t in SHour:
        prob += lpSum(I[i][t] for i in SBattery) <= param[0]
        

    for j in SCar:
        for t in SHour:
            prob += lpSum(y[(i,j)]*G[i][j][t] for i in SBattery) == Demand[t][j-1]
    


    #For Mixed
    if size==30:
        for i in SBattery:
            for t in range(0,3):
                prob += I[i][t] == StatusQuo[0][i-1]+StatusQuo[t+1][i-1]-lpSum(y[(i,j)]*G[i][j][t+2] for j in SCar)


        for i in SBattery:
            for t in range(3,size):
                prob += I[i][t] == I[i][t-1]+P[i][t-3]-lpSum(y[(i,j)]*G[i][j][t+2] for j in SCar)


        for t in SHour:
            prob += lpSum(P[i][t] for i in SBattery) <= 23
            
            
            
    #For Lot Extension
    if size==37:
        B = LpVariable.dicts("B", (SBattery, SHour), lowBound = 0, cat='Binary')
        
        for i in SBattery:
            for t in range(0,4):
                prob += I[i][t] == StatusQuoLot[0][i-1]+StatusQuoLot[t+1][i-1]-lpSum(y[(i,j)]*G[i][j][t+2] for j in SCar)


        for i in SBattery:
            for t in range(4,size):
                prob += I[i][t] == I[i][t-1]+P[i][t-4]-lpSum(y[(i,j)]*G[i][j][t+2] for j in SCar)
                
                
        for i in SBattery:
            for t in SHour:
                prob += P[i][t] == 18*B[i][t]
        for t in SHour:
            prob += lpSum(P[i][t] for i in SBattery) <= 18
            prob += lpSum(B[i][t] for i in SBattery) == 1
            
                
           
                
        
    prob += lpSum([I[i][t] for i in SBattery for t in SHour])
    status = prob.solve()
    
    inventoryRes=np.full((param[1],size), 1).tolist()
    productionRes=np.full((param[1],size), 1).tolist()
    
    for i in SBattery:
        for t in SHour:
            inventoryRes[i-1][t]=I[i][t].varValue
            productionRes[i-1][t]=P[i][t].varValue

    
    global results_inventories
        
    results_inventoriesT=[hours]+inventoryRes+productionRes
    results_inventories=np.transpose(results_inventoriesT)
        
    




    print(prob)
    print(f"status: {prob.status}, {LpStatus[prob.status]}")
    print(f"objective: {prob.objective.value()}")
    for var in prob.variables():
       print(f"{var.name}: {var.value()}")
    #for name, constraint in prob.constraints.items():
    #    print(f"{name}: {constraint.value()}")


        
    
    
####################################################################################






#################################################################################### 
    
    
    
    
def window_login():
    
    layout = [[sg.Text('Renault Battery Inventory System')],
              [sg.Text('Username:',size=(50,1)), sg.Input(size=(50,1), key='Username')],
              [sg.Text('Password:',size=(50,1)), sg.Input(size=(50,1), key='Password')],
              [sg.Button('Login')]]

    return sg.Window('Login RENMH-15', layout)


def window_choice():
    
    layout = [[sg.Text('Welcome, Please Choose Your Action')],
              [sg.Button('Battery Production Plan')],
              [sg.Button('Safety Stock Level')],
              [sg.Button('Change Parameters')],
              [sg.Button('Logout')]]

    return sg.Window('Choice of Action', layout) 




def window_production_file():
    
    working_directory=os.getcwd()
    
    layout = [[sg.Text("Choose an Excel file: ")],
              [sg.InputText(key="-FILE_PATH-"), sg.FileBrowse()],
              [sg.Button('Submit'), sg.Button('Approve'),sg.Button('Back')]]
    
    return sg.Window('Battery Production Level', layout)


def window_results():
    
    
    
    headingss=["Hours"]

    for i in range(param[1]):
        headingss.append("Inventory "+chr(i+65))

    for i in range(param[1]):
        headingss.append("Produce "+chr(i+65))
    
    
    results_inventoriess = results_inventories.tolist()

        
        
    results_window_layout = [
        [sg.Table(
        values=results_inventoriess,
        headings=headingss,
        display_row_numbers=False,
        max_col_width=35,
        auto_size_columns=True,
        justification='left',
        num_rows=10,
        key='-TABLEE-',
        row_height=35,
        tooltip="Result Table"
        )]
    ]
    
    results_window = sg.Window("RESULTS", results_window_layout, modal=True)
        


    while True:
        event, values = results_window.read()
        if event == 'Exit' or event == sg.WIN_CLOSED:
            break
    


def window_SS():
    
    layout = [[sg.Text('Desired Service Level(%)',size=(50,1))],
              [sg.Combo(values=["90","95","98","99"],default_value='95', key='service')],
              [sg.Button('Show Safety Stock Level'),sg.Button('Back')]]

    return sg.Window('Safety Stock Level', layout) 



def window_parameters():
    
    layout = [[sg.Text('Parameters',size=(50,1))],
              [sg.Text('The Capacity of the Tristock Area:',size=(50,1)), sg.Input(size=(20,1), default_text=param[0], key='tristock')],
              [sg.Text('Number of Battery Variants:',size=(50,1)), sg.Input(size=(20,1), default_text=param[1], key='numBatteries')],
              [sg.Text('Number of Car Variants:',size=(50,1)), sg.Input(size=(20,1), default_text=param[2], key='numCars')],
              [sg.Button('Change'),sg.Button('Back')]]

    return sg.Window('Parameters', layout) 


def window_compability():
    layout = [[sg.Text('Arrage the compability of batteries and cars')]+
              [[sg.Checkbox(f' ({chr(i+65)}, {j+1})', enable_events=True, key=f'{chr(i+65)}{j+1}') 
               for i in range(param[1])] for j in range(param[2])]+
              [sg.Button('Save Compability'),sg.Button('Back')]]


    return sg.Window('Compability', layout) 


####################################################################################






####################################################################################


def button_login(values):
    
    global window
    
    uname = values['Username']
    upass = values['Password']
    if uname == '':
        sg.popup('Username cannot be empty')
    elif upass == '':
        sg.popup('Password cannot be empty')
    else:
        if uname==log[0] and upass==log[1]:
            window.close()
            window = window_choice()
        else:
            sg.popup('Username or Password is wrong')
            
            
def button_production(values):
    global window
    window.close()
    window = window_production_file()

    
   

def button_submit(values):
    global data_array_demand
    global data_array_status_quo
    global data_array_status_quo0
    global hours
    global size

    excel_file_path = values["-FILE_PATH-"]
    
    if excel_file_path == '':
        sg.popup('File should be selected')
    else:
    
    
        excel_file_df = pd.read_excel(excel_file_path)
        headers_demand = excel_file_df.columns.to_numpy()[0:(param[1]+1)]
        headers_status_quo = excel_file_df.columns.to_numpy()[(param[1]+2):]
        data_array_demand = excel_file_df.to_numpy()


        size=int(int(np.size(data_array_demand))/(param[1]+param[2]+3))
        data_array_status_quo0 = np.transpose(excel_file_df.to_numpy())[-(param[2]+1):]


        if size==30:
            data_array_status_quo = np.transpose(data_array_status_quo0)[0:4]
        else:
            data_array_status_quo = np.transpose(data_array_status_quo0)[0:5]
            


        hours=[]




        for i in range(0,size):
            hour=str(data_array_demand[i][0])
            hours.append(hour)




        def create_table(name,headings, data):
            tables_window_layout = [
                [sg.Table(
                values=data,
                headings=headings,
                display_row_numbers=False,
                max_col_width=35,
                auto_size_columns=True,
                justification='right',
                num_rows=10,
                key='-TABLE-',
                row_height=35,
                tooltip="Input Table"
                )]
            ]




            tables_window = sg.Window(name, tables_window_layout, modal=True)



            while True:
                event, values = tables_window.read()
                if event == 'Exit' or event == sg.WIN_CLOSED:
                    break




        create_table("Demand", headers_demand.tolist(), data_array_demand.tolist())
        create_table("Status Quo", headers_status_quo.tolist(), data_array_status_quo.tolist())    

   
def button_approve(values):
    global window
    global demand
    global size
    global StatusQuo
    global StatusQuoLot
    window.close()
    
    size=int(int(np.size(data_array_demand))/(param[1]+param[2]+3))
    
    Init_Inventory_Lot = data_array_demand[0][-param[1]:].astype(int).tolist()
    Init_Inventory = data_array_demand[0][-param[1]:].astype(int).tolist()
    
    
    data_array_status_quoList = data_array_status_quo.tolist()[1:]
    

    for i in range(len(data_array_status_quoList)):
        if size==37:
            initProLot=data_array_status_quoList[i][1:]
            initProLotInteger=[int(h) for h in initProLot]
            Init_ProductionLot[i]=initProLotInteger
            
            
        if size==30:
            initProMix=data_array_status_quoList[i][1:]
            initProMixInteger=[int(h) for h in initProMix]
            Init_ProductionMix[i]=initProMixInteger

    
     
    StatusQuo = [Init_Inventory]+Init_ProductionMix #inventories, fist week production, second..
    StatusQuoLot = [Init_Inventory_Lot]+Init_ProductionLot #inventories, fist week production, second..
    
    #data_array_demandT = np.transpose(data_array_demand).tolist()
    
      
    
    for i in range(0,size): 
        Demand[i] = data_array_demand[i][1:param[2]+1]
        
    
    solver()

    window = window_results()
    window = window_choice()
    
    

    
    
def button_safetyStock(values):
    global window
    serviceLevel=int(values["service"])
    if serviceLevel==90:
        M1=3/24
    if serviceLevel==95:
        M1=7/24
    if serviceLevel==98:
        M1=17/24
    if serviceLevel==99:
        M1=306/24
        
    
    M2=0.52
    SSlevel=math.ceil(6*(M1+M2))
    window.close()
    sg.popup('Safety Stock Level is: '+ str(SSlevel))
    window = window_choice()
    
    
def button_change(values):
    global window
    global param
    

    if values['tristock'] == '':
        sg.popup('Tristock area cannot be empty')
    elif values['numBatteries'] == '':
        sg.popup('Number of battery variants cannot be empty')
    elif values['numCars'] == '':
        sg.popup('Number of car variants cannot be empty')
    else:
        param[0]=int(values['tristock'])
        param[1]=int(values['numBatteries'])
        param[2]=int(values['numCars'])
        
        
        para=""
        for item in param:
            para+=str(item)+" "
            
        para+="\n"

        def replace_line(file_name, line_num, text):
            lines = open(file_name, 'r').readlines()
            lines[line_num] = text
            out = open(file_name, 'w')
            out.writelines(lines)
            out.close()
            
        replace_line('parameters.txt', 0, para)
        

        
        
    sg.popup('Tristock, # of Batteries, # of Cars=', param)
    window.close()
    window = window_compability()
    
    
    
def button_compabilityUpdate(values):
    global window
    window.close()
    
    z={}
    comp_txt=[]
    
    
    def replace_line(file_name, line_num, text):
        lines = open(file_name, 'r').readlines()
        lines[line_num] = text
        out = open(file_name, 'w')
        out.writelines(lines)
        out.close()
            
        
        
        
    for i in range(param[1]):
        for j in range(param[2]):
            key=f'{chr(i+65)}{j+1}'
            if values[key]==True:
                z[(i+1,j+1)]=1
                comp_txt+=[1]
                
            else:
                z[(i+1,j+1)]=0
                comp_txt+=[0]
                    
    
    comp=""
    for item in comp_txt:
        comp+=str(item)+" "
    
    
    
    replace_line('parameters.txt', 1, comp)
    
    
    window = window_parameters()
    

    
window = window_login()

while True:
    event, values = window.read()
    
    if event == 'Login':
        button_login(values) 
        
    elif event == 'Battery Production Plan':
        window.close()
        button_production(values)
        
        
    elif event == 'Upload a File':
        window.close()
        button_file_input(values)
        
    elif event == 'Submit':
        button_submit(values)
        
    elif event == 'Approve':
        button_approve(values)           
       
    elif event == 'Safety Stock Level':
        window.close()
        window = window_SS()
    
    elif event == 'Show Safety Stock Level':
        window.close()
        button_safetyStock(values)
        
    elif event == 'Back':
        window.close()
        window = window_choice()
        
    elif event == 'Change Parameters':
        window.close()
        window = window_parameters()
        
    elif event == 'Change':
        window.close()
        button_change(values)
        
    elif event == 'Save Compability':
        window.close()
        button_compabilityUpdate(values)
        
        
    elif event == 'Logout':
        window.close()
        
        
    elif event == sg.WIN_CLOSED:
        break
    
    
    
    
    
    

{(1, 1): 1, (1, 2): 0, (1, 3): 0, (2, 1): 1, (2, 2): 0, (2, 3): 0, (3, 1): 0, (3, 2): 1, (3, 3): 1}
InvenTri:
MINIMIZE
1*I_1_0 + 1*I_1_1 + 1*I_1_10 + 1*I_1_11 + 1*I_1_12 + 1*I_1_13 + 1*I_1_14 + 1*I_1_15 + 1*I_1_16 + 1*I_1_17 + 1*I_1_18 + 1*I_1_19 + 1*I_1_2 + 1*I_1_20 + 1*I_1_21 + 1*I_1_22 + 1*I_1_23 + 1*I_1_24 + 1*I_1_25 + 1*I_1_26 + 1*I_1_27 + 1*I_1_28 + 1*I_1_29 + 1*I_1_3 + 1*I_1_30 + 1*I_1_31 + 1*I_1_32 + 1*I_1_33 + 1*I_1_34 + 1*I_1_35 + 1*I_1_36 + 1*I_1_4 + 1*I_1_5 + 1*I_1_6 + 1*I_1_7 + 1*I_1_8 + 1*I_1_9 + 1*I_2_0 + 1*I_2_1 + 1*I_2_10 + 1*I_2_11 + 1*I_2_12 + 1*I_2_13 + 1*I_2_14 + 1*I_2_15 + 1*I_2_16 + 1*I_2_17 + 1*I_2_18 + 1*I_2_19 + 1*I_2_2 + 1*I_2_20 + 1*I_2_21 + 1*I_2_22 + 1*I_2_23 + 1*I_2_24 + 1*I_2_25 + 1*I_2_26 + 1*I_2_27 + 1*I_2_28 + 1*I_2_29 + 1*I_2_3 + 1*I_2_30 + 1*I_2_31 + 1*I_2_32 + 1*I_2_33 + 1*I_2_34 + 1*I_2_35 + 1*I_2_36 + 1*I_2_4 + 1*I_2_5 + 1*I_2_6 + 1*I_2_7 + 1*I_2_8 + 1*I_2_9 + 1*I_3_0 + 1*I_3_1 + 1*I_3_10 + 1*I_3_11 + 1*I_3_12 + 1*I_3_13 + 1*I_3_14 + 1*I_3_15 + 1